In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [2]:
from cleaner import baseline_cleaner
import pandas as pd
import numpy as np

## Reading and Preparing Train Data
Baseline Modeling

In [15]:
df = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

## Train Model RF

In [16]:
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor

In [17]:
df= baseline_cleaner(df,lgbm_encode=False)
test = baseline_cleaner(test,is_test=True,lgbm_encode=False)

In [18]:
X = df.drop(['price'],axis=1)
Y = df.price

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [20]:
model = RandomForestRegressor()
model.fit(X_train,y_train)
pred = model.predict(X_test)

In [21]:
mean_squared_error(y_test,pred,squared=False)

14790.158838646574

## Train LightGBM model

In [4]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [5]:
df= baseline_cleaner(df)
test = baseline_cleaner(test,is_test=True)

In [6]:
X = df.drop(['price'],axis=1)
Y = df.price

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, random_state=42)

In [8]:
model = lgb.LGBMRegressor(
    boosting_type="gbdt",
    num_iterations = 2500,
    learning_rate = 0.05,
    num_leaves=15,
    tree_learner='feature',
    max_depth =10,
    min_data_in_leaf=7,
    bagging_fraction = 1,
    bagging_freq = 100,
    reg_sqrt='True',
    metric ='rmse',
    feature_fraction = 0.6,
    random_state=42)

model.fit(X_train,y_train) 


preds = model.predict(X_test)
rmse_lgb = mean_squared_error(y_test, preds,squared = False)
print(" RMSE: %f" % (rmse_lgb))

/Users/kanav7/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] min_data_in_leaf is set=7, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7
[LightGBM] [Warning] bagging_freq is set=100, subsample_freq=0 will be ignored. Current value: bagging_freq=100
 RMSE: 37226.634600


### Predict Values for Test Data

In [22]:
test['Predicted'] = model.predict(test)

In [23]:
test['id'] = test.index

In [24]:
submission = test[['id','Predicted']]

In [25]:
submission

,id,Predicted
0,0,146832.0
1,1,271270.0
2,2,216233.0
3,3,158709.0
4,4,62729.0
...,...,...
4995,4995,73228.0
4996,4996,169023.0
4997,4997,55658.0
4998,4998,75581.0


In [53]:
submission.to_csv('data/submission.csv',index=False)